# Nonlinear dynamics under drive

(by Dmitry R. Gulevich, ITMO University, St. Petersburg, 197101, Russia)

# 1. Model

$$
i\frac{\partial\psi_{+}}{\partial t} = -\Delta \psi_{+} + \Re[U(x,y)]\,\psi_{+} + i \Im[U(x,y)]\,\psi_{+} 
+ \beta \left(\partial_x - i \partial_y\right)^2 \psi_{-} + \left(|\psi_{+}|^2+\alpha|\psi_{-}|^2\right)\psi_+
+ \Omega\psi_+ + A_{+}e^{-i\omega t},
\\
i\frac{\partial\psi_{-}}{\partial t} = -\Delta \psi_{-} + \Re[U(x,y)]\,\psi_{-} + i \Im[U(x,y)]\,\psi_{-} 
+ \beta \left(\partial_x + i \partial_y\right)^2 \psi_{+} + \left(|\psi_{-}|^2+\alpha|\psi_{+}|^2\right)\psi_-
- \Omega\psi_- + A_{-}e^{-i\omega t},
$$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import exp2d # my module
import imp
#imp.reload(exp2d) # reload exp2d after update
from numpy.ctypeslib import ndpointer, load_library
from ctypes import *
from IPython.display import HTML

In [2]:
### === C library interface ===
libcd = load_library("liblattice.so", ".")
libcd.lattice.restype = None
libcd.lattice.argtypes = [ndpointer(dtype=np.complex128, ndim=2, flags='C_CONTIGUOUS'), 
                          ndpointer(dtype=np.complex128, ndim=2, flags='C_CONTIGUOUS'),
                          ndpointer(dtype=np.complex128, ndim=3, flags='C_CONTIGUOUS'), 
                          ndpointer(dtype=np.complex128, ndim=3, flags='C_CONTIGUOUS'),
                          c_double, c_double, c_int, c_int, c_double, c_int, c_int, 
                          ndpointer(dtype=np.complex128, ndim=2, flags='C_CONTIGUOUS'),
                          c_double, c_double, c_double, c_double, c_double, c_double, c_double, 
                          c_int, c_int]

In [3]:
### Polariton parameters
hbar=1.05e-34 # reduced Planck constant, J s
me=9.1e-31 # electron mass, kg
meff=5.0e-5*me # exciton-polariton effective mass, kg
aratio=-0.05 # nonlinearity parameter $\alpha$
beta0=0.2 # TE-TM splitting parameter
Omega=0.0 # Magnetic field

In [15]:
### Simulation parameters
M=20 # discretization of one unit cell (MxM)
Nxcells=7 # number of units cells along x
Nycells=7 # number of units cells along y
MNx=M*Nxcells
MNy=M*Nycells
Lx=1.0*Nxcells # length of the domain along x in units of lattice constant
Ly=1.0*Nycells # length of the domain along y in units of lattice constant
dt=0.0005 # simulation timestep, should be << 1/U
dtout=0.02 # output timestep
Nframes=50 # number of frames

In [4]:
### Define unit cell potential: Lieb lattice
# Lconstant = 5.8 # lattice constant, micron
# Rpillar = 1.5 # pillar radius, micron
# UinmeV=0.-0.1*1j # meV
# UoutmeV=10-0.5*1j # meV
# E0=hbar*hbar/(2.*meff*Lconstant*Lconstant*1.e-12)/(1.6e-22) # characteristic energy, meV
# U = exp2d.U_Lieb(M, Rpillar/Lconstant, UinmeV/E0, UoutmeV/E0)

In [16]:
### Define unit cell potential: square lattice
Lconstant = 5.0 # lattice constant, micron
Rpillar = 2.5 # pillar radius, micron
UinmeV=0.-0.1*1j # meV
UoutmeV=10-0.5*1j # meV
E0=hbar*hbar/(2.*meff*Lconstant*Lconstant*1.e-12)/(1.6e-22) # characteristic energy, meV
U = exp2d.U_Square(M, Rpillar/Lconstant, UinmeV/E0, UoutmeV/E0)

In [17]:
### Parameters of the pump
Rspot = 2.0 # radius of the Gaussian pump spot
i0=int(MNx/2) # location of the spot (node index along x)
j0=int(MNy/2) # location of the spot (node index along y)
A1amp=130. # drive amplitude for psi_plus component
A2amp=130. # drive amplitude for psi_minus component 
Edrive = 2.0 # pump frequency in meV
omegadrive=Edrive/E0
countout=int(dtout/dt)

In [18]:
### Define supercell potential
Uglobal=np.zeros((MNx,MNy),dtype=np.complex128)
for i in range(Nxcells):
    for j in range(Nycells):
        Uglobal[i*M:(i+1)*M,j*M:(j+1)*M] = np.copy(U[:,:])    

In [19]:
### Initialization
Psi1=np.zeros((MNx,MNy),dtype=np.complex128)
Psi2=np.zeros((MNx,MNy),dtype=np.complex128)
Psi1data=np.empty((Nframes,MNx,MNy),dtype=np.complex128)
Psi2data=np.empty((Nframes,MNx,MNy),dtype=np.complex128)

In [20]:
### Calculation using the compiled C modules
libcd.lattice(Psi1,Psi2,Psi1data,Psi2data,Lx,Ly,MNx,MNy,dt,Nframes,countout,Uglobal,
              aratio,beta0,Omega,A1amp,A2amp,omegadrive,Rspot,i0,j0)

In [21]:
### Display the last frame
#exp2d.cpsi12(Psi1data[-1],Psi2data[-1],interpolation='gaussian')

In [22]:
### Create animation
anim=exp2d.vipsi12(Psi1data,Psi2data,interpolation='gaussian')

In [23]:
### Display animation
HTML(anim.to_html5_video())

In [14]:
### Save animation to file
#anim.save('square.mp4', writer = 'ffmpeg', fps=15)